In [ ]:
!pip install torch transformers langchain python-dotenv openai google-api-python-client

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

# OpenAI Version

In [ ]:
from langchain.agents import tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_function
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor


In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def get_text_word_count(sentence: str) -> int:
    """Returns the number of words in a sentence."""
    return len(sentence.split())


wiki_search = GoogleSearchAPIWrapper()

@tool
def search_wikipedia(query: str) -> str:
    """Searches Wikipedia for a query."""
    return wiki_search.run(query)

toolbox = {
    "get_word_length": get_word_length,
    "get_text_word_count": get_text_word_count,
    "search_wikipedia": search_wikipedia,
}

In [ ]:
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in toolbox.values()])

In [ ]:
system_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | system_prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=list(toolbox.values()), verbose=True)

# QUESTION = "Can you count the number of letters in the word anticonstitutionnellement?"
QUESTION = "Can you get the name of the COP28 host country?"

agent_executor.invoke({"input": QUESTION})


# OS Version

In [ ]:
from langchain.llms.huggingface_endpoint import HuggingFaceEndpoint

endpoint_hermes = 'https://mwgkynxou5oy7i3j.us-east-1.aws.endpoints.huggingface.cloud'

llm = HuggingFaceEndpoint(endpoint_url=endpoint_hermes, task='text-generation', model_kwargs={"do_sample" : False, "max_new_tokens" : 100})

In [ ]:

import asyncio
from typing import List, Union

from langchain_core.agents import AgentAction, AgentActionMessageLog, AgentFinish
from langchain_core.messages import (
    AIMessage,
)
from langchain_core.outputs import ChatGeneration, Generation
from langchain.schema.messages import AIMessage
from langchain.agents.agent import AgentOutputParser
import ast

def extract_function_call_hermes(call: str):
    """Extracts function name and arguments from a text function call"""
    function_call = ast.literal_eval(call)
    return function_call["name"], function_call["arguments"]


class HuggingFaceFunctionsAgentOutputParser(AgentOutputParser):
    """Parses a message into agent action/finish.

    Is meant to be used with HF models.
    """
    @property
    def _type(self) -> str:
        return "huggingface-functions-agent"

    def parse(self, message: str) -> Union[AgentAction, AgentFinish]:
        """
        Parse a message into function call or final output
        """
        function_call_begin_marker ="<functioncall>"
        function_call_end_marker ="<functionresp>"

        function_call = function_call_begin_marker in message

        if function_call:
            message = message.replace('\n', '')
            marker_begin, marker_end = function_call_begin_marker, function_call_end_marker
            begin = message.find(marker_begin) + len(marker_begin)
            # end = message.find(marker_end)
            end=-1
            if end == -1:
                end = message.find('<|im_end|>')

            if end == -1:
                end = message.find('}}') + 2
            # If no end was found
            if end == -1:
                call_text = message[begin:]
            else:
                call_text = message[begin:end]
            call_text = call_text.strip()

            function_name, tool_input = extract_function_call_hermes(call_text)
            
            content_msg = f"responded: {message}\n" if message else "\n"
            log = f"\nInvoking: `{function_name}` with `{tool_input}`\n{content_msg}\n"
            return AgentActionMessageLog(
                tool=function_name,
                tool_input=tool_input,
                log=log,
                message_log=[AIMessage(content=message)],
            )

        return AgentFinish(
            return_values={"output": message}, log=str(message)
        )


### Provisory:
- message handled as a string in the InputParser

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from prompt_formats import prompt_hermes

system_prompt = prompt_hermes

system_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
    ]
)

In [ ]:
tool_descriptions = [format_tool_to_openai_function(t) for t in toolbox.values()]

In [ ]:
llm_with_tools = llm.bind(functions=tool_descriptions)

agent = (
    {
        "input": lambda x: x["input"]
    }
    | system_prompt
    | llm_with_tools
    | HuggingFaceFunctionsAgentOutputParser()
)

In [ ]:
from langchain.schema.agent import AgentFinish

# QUESTION = "Can you count the letters in word 'bonjourmonami'?" 
# QUESTION = "Can you extend the sentence 'Hello Sir, the weather is terrible' to 10 words? Check that your output sentence has exactly 10 words!"
# QUESTION = "Can you get the number of words in the sentence 'Hello Sir, terrible weather'?"
QUESTION = "Can you get the name of the COP28 host country? Only answer with the country name."
# QUESTION = "Who are the pitchers with the number before and after Taishō Tamai's number as of July 2023? Give them to me in the form Pitcher Before, Pitcher After, use their last names only, in Roman characters."

intermediate_steps = []
history=''
while True:
    output = agent.invoke(
        {
            "input": QUESTION ss+ f" Available functions are {tool_descriptions}. For the record, {history}. Call a function only if needed.",
            "intermediate_steps": intermediate_steps,
        }
    )
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        tool = toolbox[output.tool]
        observation = tool.run(output.tool_input)
        print(f"Calling {output.tool} with input {output.tool_input} returned: {observation}")
        history+=f"calling {output.tool} with input {output.tool_input} returned: {observation} ;"
print("AGENT ANSWER:::", final_result)

# Benchmarking